<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  py_booked_amt = 304000 + 1007000
  cy_booked_amt = 377000 + 297000 + 55000
  reserve_amt =  1000
  total_booked_amt = py_booked_amt + cy_booked_amt
  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'TFrame', 'LatestQtr', 'StarStock', 'BizFinTech', 'Conviction', 'Category', 'InFolio', 'XIRR', 'Remarks']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2024-10-04 11:01:44
-------------------
qualified stocks: 86
with latest results: 82
still star stocks: 40
-------------------
Initial Investment:  95.12 L
CY Investment:  1.08 C
Reserve:  1.00 K
Current:  1.22 C
-------------------
Today PnL: 28.19 K (0.23%)
Current PnL: 6.47 L (5.98%)
CY Booked + Current PnL: 13.76 L (12.72%)
-------------------
Total profit:  11.82 L
Total loss:  -5.35 L
-------------------
Total Booked + Current PnL: 26.87 L (28.25%)
Total Booked PnL: 20.40 L (21.45%)
Curr Year Booked PnL: 7.29 L (5.96%)
Prev Year Booked PnL: 13.11 L (13.78%)
Est FTT:  1.83 C
Est FTT PnL: 60.96 L (49.84%)
-------------------
Est LTT:  2.31 C
Est LTT PnL: 1.08 C (88.48%)
Deployed:  95.12 L
Current:  1.22 C
CAGR/XIRR %: 26.81%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'Close', 'FTT', 'Dev%_PE', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'XIRR', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost    Close     FTT  Dev%_PE    RSP   Current  \
49  LALPATHLAB  2519.90  3483.15  3689.0     7.66  77.91  125393.0   
33     HCLTECH  1349.00  1797.95  2006.0    42.99  66.28  134846.0   
51        LTIM  5051.49  6248.00  7286.8    26.48  82.56   99968.0   
57  PIDILITIND  2482.94  3260.00  3576.0     0.23  69.77  189080.0   
43      JIOFIN   320.24   343.35   387.0    -1.28  54.65  169615.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%   XIRR  \
49      34677.0   7411.0        1.87         38.23   5.91  46.39   51.0   
33      33671.0  15602.0        1.09         33.28  11.57  48.70  128.0   
51      19144.0  16625.0        1.04         23.69  16.63  44.25   22.0   
57      45069.0  18322.0       -1.21         31.30   9.69  44.02   27.0   
43      11416.0  21558.0       -0.65          7.22  12.71  20.85  139.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
49     4.68        1.03    73.89     X40N      NTT  HEALTHCARE  
33     2.16        1.10    52.64      X40      BTT          IT  
51     1.15        0.82    37.23     X200      ATH          IT  
57     2.46        1.55    42.74      X40      BTT   CHEMICALS  
43     0.53        1.39    66.19      XSP      BTT     FINANCE

In [10]:
# top 5 for average up
tmp_df = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 15) & (df_portfolio_features['FTT%'] > 15) ].sort_values(by = 'RSP', ascending=False)
tmp_df[cols].head()

Symbol  AvgCost    Close      FTT  Dev%_PE     RSP   Current  \
44  JMFINANCIL    82.95   146.66   172.00   177.28  100.00  172619.0   
55  METROPOLIS  1399.27  2279.25  3102.00    51.17   93.02  152710.0   
22       CLEAN  1389.74  1602.00  2018.00     7.18   90.70  131364.0   
5     ANGELONE  2246.65  2700.15  3791.17    -5.25   89.53  189010.0   
13  BANKBARODA   195.74   251.92   395.00   -28.65   84.88   99508.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%    OTT%   XIRR  \
44      74987.0  29829.0        2.02         76.81  17.28  107.35   96.0   
55      58959.0  55128.0        1.60         62.89  36.10  121.69   46.0   
22      17405.0  34115.0        0.49         15.27  25.97   45.21   10.0   
5       31744.0  76379.0       -1.53         20.19  40.41   68.75  106.0   
13      22191.0  56521.0        2.80         28.70  56.80  101.80   20.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
44     2.51        1.41   109.19       XR      NTT     FINANCE  
55     1.07        1.25    65.06       XR      NTT  HEALTHCARE  
22     0.51        1.07    25.57       XR      NTT   CHEMICALS  
5      0.42        1.55    37.85     X40N      ATH     FINANCE  
13     0.39        0.81    36.91       XR      BTT       BANKS

In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost    Close     FTT  Dev%_PE     RSP   Current  \
52      LUXIND  1448.19  2240.70  2880.0    85.53   16.28  212866.0   
84      VIPIND   488.80   546.80   718.0      NaN   95.35  106626.0   
13  BANKBARODA   195.74   251.92   395.0   -28.65   84.88   99508.0   
40        INFY  1436.24  1934.35  2275.0    11.85   65.12  201172.0   
44  JMFINANCIL    82.95   146.66   172.0   177.28  100.00  172619.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%    OTT%  XIRR  \
52      75288.0  60731.0        6.10         54.72  28.53   98.87  41.0   
84      11310.0  33385.0        4.63         11.87  31.31   46.89   NaN   
13      22191.0  56521.0        2.80         28.70  56.80  101.80  20.0   
40      51803.0  35426.0        2.16         34.68  17.61   58.40  47.0   
44      74987.0  29829.0        2.02         76.81  17.28  107.35  96.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
52     1.24        1.74   108.75     X40N      NTT  TEXTILES  
84     0.34        0.87    26.54     X40N      NTT      MISC  
13     0.39        0.81    36.91       XR      BTT     BANKS  
40     1.46        1.64    44.88      X40      BTT        IT  
44     2.51        1.41   109.19       XR      NTT   FINANCE

In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost    Close     FTT  Dev%_PE    RSP   Current  \
34    HDFCBANK  1549.96  1682.00  2170.0   -11.04  80.23  353220.0   
71    SONACOMS   578.05   690.55  1006.0    -4.70  27.91  120846.0   
17  BERGEPAINT   558.54   584.50   680.0   -22.84  39.53  185286.0   
74    SURYODAY   189.78   170.86   240.0   -34.20   8.14  134979.0   
67    SBIETFPB   241.24   264.18   400.0      NaN  67.44  199984.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  XIRR  \
34      27728.0  102469.0       -2.56          8.52  29.01  40.00  11.0   
71      19687.0   55202.0       -2.27         19.46  45.68  74.03  41.0   
17       8229.0   30276.0       -2.10          4.65  16.34  21.75  10.0   
74     -14947.0   54626.0       -2.04         -9.97  40.47  26.46  -5.0   
67      17365.0  102812.0       -1.84          9.51  51.41  65.81  26.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
34     0.27        2.89    23.19      X40      BTT    BANKS  
71     0.36        0.99    36.33       AR      BTT     AUTO  
17     0.27        1.52    29.09      XSP      NTT   PAINTS  
74    -0.27        1.10    16.11       XR      NTT    BANKS  
67     0.17        1.64    18.96      XSP      BTT    BANKS

In [13]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost    Close      FTT  Dev%_PE     RSP   Current  \
44  JMFINANCIL    82.95   146.66   172.00   177.28  100.00  172619.0   
20      CAMPUS   294.86   329.50   631.85    26.67   98.84  187486.0   
42       JCHAC  2143.67  2450.45  3542.00      NaN   97.67   90667.0   
45     JPPOWER    18.73    20.15    26.20   -59.08   96.51  152596.0   
84      VIPIND   488.80   546.80   718.00      NaN   95.35  106626.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%    OTT%   XIRR  \
44      74987.0   29829.0        2.02         76.81  17.28  107.35   96.0   
20      19711.0  172037.0        1.42         11.75  91.76  114.29  187.0   
42      11351.0   40383.0       -0.90         14.31  44.54   65.23    NaN   
45      10754.0   45809.0        0.25          7.58  30.02   39.88  133.0   
84      11310.0   33385.0        4.63         11.87  31.31   46.89    NaN   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
44     2.51        1.41   109.19       XR      NTT   FINANCE  
20     0.11        1.53    53.61      XSP      ATH  FOOTWEAR  
42     0.28        0.74   156.83     X40N      BTT        AC  
45     0.23        1.25   130.29      XSP      NTT     POWER  
84     0.34        0.87    26.54     X40N      NTT      MISC

In [14]:
# top 5 of lower current allocation with minimal upside
df_tmp = df_portfolio_features[(df_portfolio_features['CurrAlloc%'] < 1) ].sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost    Close     FTT  Dev%_PE    RSP   Current  \
51        LTIM  5051.49  6248.00  7286.8    26.48  82.56   99968.0   
39  INDUSINDBK  1354.50  1407.30  1800.0   -20.42  44.19   80216.0   
24       DABUR   523.35   580.85   735.0    -1.19   5.81  101649.0   
16   BAYERCROP  4346.57  6518.30  8325.0    25.75  61.63  104293.0   
8     ATULAUTO   656.09   658.00   844.0      NaN  12.79  103964.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  XIRR  \
51      19144.0  16625.0        1.04         23.69  16.63  44.25  22.0   
39       3010.0  22380.0        1.41          3.90  27.90  32.89   8.0   
24      10063.0  26978.0        0.11         10.99  26.54  40.44  31.0   
16      34748.0  28910.0        0.33         49.96  27.72  91.53  33.0   
8         302.0  29391.0        1.12          0.29  28.27  28.64   NaN   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
51     1.15        0.82    37.23     X200      ATH         IT  
39     0.13        0.66     5.17       XR      NTT      BANKS  
24     0.37        0.83    17.80      X40      BTT       FMCG  
16     1.20        0.85    37.54     X40N      BTT  CHEMICALS  
8      0.01        0.85    38.32      XSP      NTT       AUTO

In [15]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 22.0 seconds
